In [1]:
from ocr import OCR
from PIL import Image
import cv2
import os
import keras_ocr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob as glob
import openpyxl as op
import re
from joblib import Parallel, delayed
import joblib
import uuid
import keras_ocr
import plotly.express as px
import tqdm

import pickle
import easyocr
import pytesseract


2023-02-20 01:22:12.533174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 01:22:13.367885: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ubuntu/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-02-20 01:22:13.367965: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ubuntu/anaconda3/lib/python3.9/site-packages/cv2/../../l

In [48]:
def get_box_densities(idx):
    file=files[idx]
    Image.open(file)
    cropped = Image.open(file).crop(
    (1530, 0, 1850, 900)
    )
    cropped=np.array(cropped)
    reader=easyocr.Reader(['en'])
    allimg=reader.readtext(cropped,batch_size=20)
    box=[ allimg[line] for line in range(len(allimg)) if 'Whole' in allimg[line][1] ] 
    return idx,box

def get_box_faz(idx):
    file=files[idx]
    Image.open(file)
    cropped = Image.open(file).crop(
    (330,60,1300,500)
    )
    cropped=np.array(cropped)
    reader=easyocr.Reader(['en'])
    allimg=reader.readtext(cropped,batch_size=20,width_ths=1.5)
    box=[ allimg[line] for line in range(len(allimg)) if ('mm' in allimg[line][1]) or ('FD' in allimg[line][1]) or ('Scan Quality' in allimg[line][1]) ] 
    return idx,box

def get_dens_qualities(idx):
    file=files[idx]
    cropped_qual = Image.open(file).crop(
    (900, 0, 1250, 650)
    )
    cropped=np.array(cropped_qual)
    reader=easyocr.Reader(['en'])
    ocr_qual=reader.readtext(cropped,batch_size=20)
    qual_elem=[elem for elem in ocr_qual if 'Scan' in elem[1]][-1][1] #-q is if it has more
    qual_value=extract_score(qual_elem)[0]
    return idx,qual_value

def flatten(l):
    return [item for sublist in l for item in sublist]

def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False
    
def extract_score(text):
    match = re.search(r'(\d+)/(\d+)', text)
    if match:
        return match.group(0)
    return None


def infer_regex(arr):
    # Convert floats to strings and remove trailing zeros
#     str_arr = [str(num).rstrip("0").rstrip(".") for num in arr]
    str_arr = [str(num) for num in arr]
    # Count the number of occurrences of each format
    format_counts = Counter([re.sub(r"\d", "d", s) for s in str_arr])
    
    # Sort formats by popularity and length
    formats = sorted(format_counts.keys(), key=lambda f: (-format_counts[f], -len(f)))
    ######################################################################################
    #######  HERE GOES IF YOU WANT TO TREAT MULTIPLE POPULATIONS OF POSSIBLE FORMATS ##### 
    ######################################################################################
    print(format_counts)
    if '.' in formats[0]:
        pre,post=formats[0].split('.')
        lpre=len(pre)
        lpost=len(post) 
        format_regex= f"^\d{{{lpre}}}.\d{{{lpost}}}$"
    else:
        pre=formats[0]
        post=0
        lpre=len(pre)
        format_regex= f"^\d{{{lpre}}}$"
    # Use the most popular and most stringent format to generate a regex
    
#     format_regex = formats[0].replace("d", "\\d")
#     regex = "^" + format_regex + "$"
    
    return format_regex

# DICT CROPS

In [110]:
crops={
    'densities':{},
    'faz':{}
      }
crops['densities']['table_top_left']={
                                'l' : 1530,
                                't' : 154,
                                'r' : 1600,
                                'b' : 540
                                }
crops['densities']['table_top_right']={
                                'l' : 1750,
                                't' : 154,
                                'r' : 1825,
                                'b' : 540
                                }
crops['densities']['table_bot']={
                                'l' : 1535,
                                't' : 560,
                                'r' : 1825,
                                'b' : 630
                                }
crops['densities']['quality']={
                                'l' : 1116,
                                't' : 55,
                                'r' : 1200,
                                'b' : 100
                                }
crops['faz']['values']={
                            'l' : 400,
                            't' : 90,
                            'r' : 800,
                            'b' : 125
                            }


crops['faz']['quality']={
                        'l' : 1116,
                        't' : 55,
                        'r' : 1200,
                        'b' : 100
                        }
shift=125

# FILES

In [3]:
folders_patients=glob('../data/NOPAIN_OCTA/P*')
folders_visits=glob('../data/NOPAIN_OCTA//P*/V*')
visits=sorted(
    list(
        set([folder.split("/")[4] for folder in folders_visits])
    )
)
patients=sorted(
    list(
        set([folder.split('/')[3] for folder in folders_patients]) 
    )
)
#second iteration
files=glob('../data/NOPAIN_OCTA//*/*/*ng')
files=[file for file in files if 'QuickVue' not in file]
# files=[file for file in files if ('Extra' not in file) and ('Additional' not in file)]
imgpervisit={}
for visit in visits:
    imgpervisit[visit]=sorted([file for file in files if visit in file])
    

# DEAL WITH DENSITIES

In [5]:
files=[file for file in files if ('Extra' not in file) and ('Additional' not in file) and ('(2)' not in file)]
files=[file for file in files if ('Density.png' not in file) and ('Density.Png') not in file]
files=sorted(files)
files

['../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 3.0 - Density DP.png',
 '../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 3.0 - Density SUP.png',
 '../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 3.0 - FAZ.png',
 '../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 6.0 - Density DP.png',
 '../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 6.0 - Density SUP.png',
 '../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 6.0 - FAZ.png',
 '../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 3.0 - Density DP.png',
 '../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 3.0 - Density SUP.png',
 '../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 3.0 - FAZ.png',
 '../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 6.0 - Density DP.png',
 '../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 6.0 - Density SUP.png',
 '../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 6.0 - FAZ.png',
 '../data/NOPAIN_OCTA/P001/V3/P001, R eye, V3, 3.0 - Density DP.png',
 '../data/NOPAIN_OCTA/P001/V3/P001, R eye, V3, 3.0 - Density SUP.png',
 '../data/NOPAIN_OCTA/P001/V3/P001, R eye, 

In [6]:
#FIND ONLY DENSITIES
idx_dens=[]
for i,file in enumerate(files):
    if 'Density' in file:
        print(i,file)
        idx_dens.append(i)

0 ../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 3.0 - Density DP.png
1 ../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 3.0 - Density SUP.png
3 ../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 6.0 - Density DP.png
4 ../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 6.0 - Density SUP.png
6 ../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 3.0 - Density DP.png
7 ../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 3.0 - Density SUP.png
9 ../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 6.0 - Density DP.png
10 ../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 6.0 - Density SUP.png
12 ../data/NOPAIN_OCTA/P001/V3/P001, R eye, V3, 3.0 - Density DP.png
13 ../data/NOPAIN_OCTA/P001/V3/P001, R eye, V3, 3.0 - Density SUP.png
15 ../data/NOPAIN_OCTA/P001/V3/P001, R eye, V3, 6.0 - Density DP.png
16 ../data/NOPAIN_OCTA/P001/V3/P001, R eye, V3, 6.0 - Density SUP.png
18 ../data/NOPAIN_OCTA/P002/V2/P002, L eye, V2, 3.0 - Density DP.png
19 ../data/NOPAIN_OCTA/P002/V2/P002, L eye, V2, 3.0 - Density SUP.png
21 ../data/NOPAIN_OCTA/P002/V2/P00

## GET BOXES RELATIVE POSITION WITH EASYOCR

In [121]:
%%time
boxes=Parallel(n_jobs=-1)(delayed(get_box_densities)(idx)for idx in idx_dens)
quals=Parallel(n_jobs=-1)(delayed(get_dens_qualities)(idx)for idx in idx_dens)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

...

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CPU times: user 15.9 s, sys: 1.79 s, total: 17.7 s
Wall time: 14min 42s


## CLEAN LIST

Some images are wrong in that they don't have a box with data (see Density 8.0 etc). We gotta remove these files from the list. Also there are images with two boxes of data. In that case we gotta keep the bottom box, which because Tesseract reads from top down, it's the last box.

In [124]:
boxes_clean={}

for box in boxes:
    try:
        el=box[1][0][1]
        if 'Whole' in el and len(el)>1:                                 #If Whole is found it means that there is a box
            boxes_clean[box[0]]=[box[1][-1][0],box[1][-1][1]] #Keep the last box where there are >1 box
        elif 'Whole' in el and len(el)==1:
            boxes_clean[box[0]]=[box[1][0][0],box[1][0][1]]
        elif 'Whole' not in el:                                        #If Whole is not found, then it is one of those erroneous files
            print(i,files[i],'error')
    except:
        print(box,files[box[0]],'\tWas eliminated from the analysis')   #If something fails show me where end eliminate it.

(912, []) ../data/NOPAIN_OCTA/P053/V3/P053, L eye, V3, 8.0 - Density DP.png 	Was eliminated from the analysis
(913, []) ../data/NOPAIN_OCTA/P053/V3/P053, L eye, V3, 8.0 - Density SUP.png 	Was eliminated from the analysis
(917, []) ../data/NOPAIN_OCTA/P053/V3/P053, R eye, V3, 8.0 - Density DP.png 	Was eliminated from the analysis
(918, []) ../data/NOPAIN_OCTA/P053/V3/P053, R eye, V3, 8.0 - Density SUP.png 	Was eliminated from the analysis
(1304, []) ../data/NOPAIN_OCTA/P076/V2/P076, R eye, V2, 6.0 - Density DP.png 	Was eliminated from the analysis


In [125]:
qualities={}
for qual in quals:
    qualities[qual[0]]=qual[1]

### SAVE EM

In [7]:
# with open('boxes_dens.pickle', 'wb') as handle:
#     pickle.dump(boxes_clean, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('qualities_dens.pickle', 'wb') as handle:
#     pickle.dump(qualities, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # OR IMPORT THEM
boxes_clean = pickle.load( open("boxes_dens.pickle",'rb'))
qualities= pickle.load(open("qualities_dens.pickle",'rb'))

# CROP AND READ IMAGES

In [8]:
# TAKE THE CORNERS FROM ALL BOXES FOUND BY THE OCR
corner1={}
corner2={}
corner3={}
corner4={}
for box in boxes_clean:
    corner1[box]=boxes_clean[box][0][0] 
    corner2[box]=boxes_clean[box][0][1] 
    corner3[box]=boxes_clean[box][0][2] 
    corner4[box]=boxes_clean[box][0][3] 

In [133]:
%%time
densities_data={}
errors_dens={}
for box in boxes_clean:
    errors_dens[box]={}
    file=files[box]
    cropped1 = Image.open(file).crop( 
     (1530, 0, 1850, 900)
    )                             #now take the slice of the image that we read before
    
    #LEFT
    cropped_top_left=cropped1.crop(
        (corner1[box][0]-70,
         corner1[box][1],
         corner3[box][0]-100,
         corner3[box][1]+362)
    ) #now take coords from top left and bot right to crop,so corner 1 and corner 3 for the left column
    
    ocr_tess_left=pytesseract.image_to_string(cropped_top_left,config ='-c tessedit_char_whitelist=0123456789.nN/A --psm 6')
    ocr_tess_left=ocr_tess_left.replace('.','').replace(',','').replace('(','').replace('l','1').replace('°','')
    lst_tess_left=ocr_tess_left.replace('\x0c','').split('\n')[:-1]
    lst_tess_left=[elem[0:2]+'.'+elem[2:]  if isfloat(elem) else elem for elem in lst_tess_left]
    
    #check for errors_dens
    _=[elem for elem in lst_tess_left if not isfloat(elem) and not elem=='N/A']
    if len(_) >0 :
        errors_dens[box]['top_left']=_
        
    #RIGHT
    cropped_top_right=cropped1.crop(
        (corner1[box][0]+150,
         corner1[box][1],
         corner3[box][0]+110,
         corner3[box][1]+362)
    ) #now take coords from top left and bot right to crop,so corner 1 and corner 3 for the right column

    ocr_tess_right=pytesseract.image_to_string(cropped_top_right,config ='-c tessedit_char_whitelist=0123456789.nN/A --psm 6')
    ocr_tess_right=ocr_tess_right.replace('.','').replace(',','').replace('(','').replace('l','1').replace('°','')
    lst_tess_right=ocr_tess_right.replace('\x0c','').split('\n')[:-1]
     
    _=[elem for elem in lst_tess_right if not isfloat(elem) and not elem=='N/A']
    if len(_) >0 :
        errors_dens[box]['top_right']=_
        
    
    #BOTTOM
    cropped_bottom=cropped1.crop(
        (corner1[box][0]-70,
         corner1[box][1]+405,
         corner3[box][0]+110,
         corner3[box][1]+450)
    )
    try:
        ocr_tess_bottom=pytesseract.image_to_string(cropped_bottom,config ='-c tessedit_char_whitelist=0123456789.nN/A --psm 6')
        ocr_tess_bottom=ocr_tess_bottom.replace('.','').replace(',','').replace('l','1').replace('(','').replace('°','')
        lst_tess_bottom=ocr_tess_bottom.replace('\x0c','').split('\n')[:-1]
        lst_tess_bottom=flatten([elem.split() for elem in lst_tess_bottom])
        lst_tess_bottom=[elem[0:2]+'.'+elem[2:]  if isfloat(elem) else elem for elem in lst_tess_bottom]
        
        _=[elem for elem in lst_tess_bottom if not isfloat(elem) and not elem=='N/A']
        if len(_) >0 :
            errors_dens[box]['bottom']=_
        
        if not errors_dens[box]:
            del errors_dens[box]
        
    except Exception as e:
        print(e, i,files[box])
    densities_data[box]={
        'top_left':lst_tess_left,
        'top_right':lst_tess_right,
        'bottom':lst_tess_bottom
    }


CPU times: user 36.8 s, sys: 47.8 s, total: 1min 24s
Wall time: 8min 46s


In [134]:
import copy
densities_bak=copy.deepcopy(densities_data)

### ERRORS CHECK 

In [1]:
print("The following files have an error in the character recognition. Reading an A instead of a 4 is a common error")
all_errors_dens={}
for idx in errors_dens:
    try:
        where=list(errors_dens[idx].keys())[0]
        which=densities_data[idx][where].index(errors_dens[idx][where][0])
    except:
        pass

    if 'A' in densities_data[idx][where][which]:
        corr=densities_data[idx][where][which].replace('A','4')
        corr=corr[0:2]+'.'+corr[2:]
        if isfloat(corr)and len(corr)>=3:
            densities_data[idx][where][which]=corr
        else:
            pass

        all_errors_dens[idx]="minor"
        print(densities_data[idx][where])
        
print("\nThe following files have unidentified problems, most likely the OCR missed a value or the image is corrupted")        
for idx in densities_data:
    if len(densities_data[idx]['top_left'])!=18 or len(densities_data[idx]['top_right'])!=18 or len(densities_data[idx]['bottom'])!=9:
        print(idx,files[idx])
        all_errors_dens[idx]="major"
        
len(all_errors_dens)

The following files have an error in the character recognition. Reading an A instead of a 4 is a common error


NameError: name 'errors_dens' is not defined

idx 60 is a file in which the toggle windows overlapped with a value
idx 389 and 397 are probably a result of the regex that I use to correct, cause the values of perifovea are one super low (CAN BE CORRECTED)
idx 574 is a file with no values in it (but passed through the first filter cause Whole is on the top window)

NOW WE HAVE ALL THE DENSITIES DATA IN densities_data WITH THE INDEX LINKING TO THE FILES THROUGH indices_clean.

### SAVE EM 

In [9]:
# with open('densities_data.pickle', 'wb') as handle:
#     pickle.dump(densities_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('all_errors_dens.pickle', 'wb') as handle:
#     pickle.dump(all_errors_dens, handle, protocol=pickle.HIGHEST_PROTOCOL)

# #OR LOAD THEM
densities_data = pickle.load( open("densities_data.pickle",'rb'))
all_errors_dens= pickle.load(open("all_errors_dens.pickle",'rb'))



# FAZ

In [145]:
#FIND ONLY DENSITIES
idx_faz=[]
for i,file in enumerate(files):
    if 'FAZ' in file:
        print(i,file)
        idx_faz.append(i)

2 ../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 3.0 - FAZ.png
5 ../data/NOPAIN_OCTA/P001/V2/P001, L eye, V2, 6.0 - FAZ.png
8 ../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 3.0 - FAZ.png
11 ../data/NOPAIN_OCTA/P001/V2/P001, R eye, V2, 6.0 - FAZ.png
14 ../data/NOPAIN_OCTA/P001/V3/P001, R eye, V3, 3.0 - FAZ.png
17 ../data/NOPAIN_OCTA/P001/V3/P001, R eye, V3, 6.0 - FAZ.png
20 ../data/NOPAIN_OCTA/P002/V2/P002, L eye, V2, 3.0 - FAZ.png
23 ../data/NOPAIN_OCTA/P002/V2/P002, R eye, V2, 3.0 - FAZ.png
26 ../data/NOPAIN_OCTA/P002/V2/P002, R eye, V2, 6.0 - FAZ.png
29 ../data/NOPAIN_OCTA/P002/V3/P002, L eye, V3, 3.0 - FAZ.png
32 ../data/NOPAIN_OCTA/P002/V3/P002, L eye, V3, 6.0 - FAZ.png
35 ../data/NOPAIN_OCTA/P002/V3/P002, R eye, V3, 3.0 - FAZ.png
38 ../data/NOPAIN_OCTA/P002/V3/P002, R eye, V3, 6.0 - FAZ.png
41 ../data/NOPAIN_OCTA/P003/V2/P003, L eye, V2, 3.0 - FAZ.png
44 ../data/NOPAIN_OCTA/P003/V2/P003, L eye, V2, 6.0 - FAZ.png
47 ../data/NOPAIN_OCTA/P003/V2/P003, R eye, V2, 3.0 - FAZ.png
50 ../data/

## FIND BOXES

In [146]:
boxes_faz=Parallel(n_jobs=-1)(delayed(get_box_faz)(idx)for idx in idx_faz)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


## CLEAN RESULTS

In [150]:
boxes_faz_clean={}
for box in boxes_faz:  
    boxes_faz_clean[box[0]]={}
    for info in box[1]:
        if 'Scan Quality' in info[1] :
            _qual=[info[0],info[1]]
        elif 'FAZ' in info[1]:
            _faz=[info[0],info[1]]
    boxes_faz_clean[box[0]]['qual']=_qual
    boxes_faz_clean[box[0]]['faz']=_faz

#LENGHT ERROR CHECK
for box in boxes_faz_clean.items():
    if len(box[1]['faz'][0])!=4: 
        print(box)

### SAVE EM

In [10]:
# with open('boxes_faz_clean.pickle', 'wb') as handle:
#     pickle.dump(boxes_faz_clean, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# OR LOAD EM 
boxes_faz_clean = pickle.load( open("boxes_faz_clean.pickle",'rb'))

## CORNERS

In [11]:
# TAKE THE CORNERS FROM ALL boxes_faz FOUND BY THE OCR
corner_faz1={}
corner_faz2={}
corner_faz3={}
corner_faz4={}
for box in boxes_faz_clean:
    corner_faz1[box]={'qual':boxes_faz_clean[box]['qual'][0][0],'faz':boxes_faz_clean[box]['faz'][0][0]}   #top left
    corner_faz2[box]={'qual':boxes_faz_clean[box]['qual'][0][1],'faz':boxes_faz_clean[box]['faz'][0][1]}   #top right
    corner_faz3[box]={'qual':boxes_faz_clean[box]['qual'][0][2],'faz':boxes_faz_clean[box]['faz'][0][2]}      #bot right
    corner_faz4[box]={'qual':boxes_faz_clean[box]['qual'][0][3],'faz':boxes_faz_clean[box]['faz'][0][3]}       #bot left

# CROP AND READ

In [153]:
faz_data={}
errors_faz={}
for box in boxes_faz_clean:

    errors_faz[box]={}
 
    cropped1 = Image.open(files[box]).crop( 
     (330,60,1300,500)
    )                            
    
#####FAZ
    cropped_faz=cropped1.crop(
        (corner_faz1[box]['faz'][0],
         corner_faz1[box]['faz'][1],
         corner_faz3[box]['faz'][0],
         corner_faz3[box]['faz'][1])
    )
    
    ocr_tess_faz=pytesseract.image_to_string(cropped_faz,config ='--psm 6')
    lst_tess_faz=re.findall(r'\b\d+(?:\.\d+)?\b', ocr_tess_faz)
    #check for errors_faz
    if len(lst_tess_faz) !=3 :
        lst_tess_faz=re.findall(r'\bN/A\b', ocr_tess_faz)
        if len(lst_tess_faz) !=3:
            errors_faz[box]['faz']=ocr_tess_faz
        
######QUALITY
    cropped_qual=cropped1.crop(
            (corner_faz1[box]['qual'][0],
             corner_faz1[box]['qual'][1],
             corner_faz3[box]['qual'][0],
             corner_faz3[box]['qual'][1])
    )

    ocr_tess_qual=pytesseract.image_to_string(cropped_qual,config ='--psm 6')
    tess_qual=extract_score(ocr_tess_qual)[0]
   
   
    if not isfloat(tess_qual) :
        errors_faz[box]['qual']=ocr_tess_qual
        
    
    faz_data[box]={
        'faz':lst_tess_faz,
        'qual':tess_qual,
        }

In [156]:
with open('faz_data.pickle', 'wb') as handle:
    pickle.dump(faz_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('errors_faz.pickle', 'wb') as handle:
    pickle.dump(errors_faz, handle, protocol=pickle.HIGHEST_PROTOCOL)


# LOAD BOXES

In [157]:
glob('*pickle')

['faz_data.pickle',
 'errors_faz.pickle',
 'densities_data.pickle',
 'boxes_dens.pickle',
 'boxes_faz_clean.pickle',
 'all_errors_dens.pickle',
 'qualities_dens.pickle']

In [24]:
boxes_dens=pickle.load(open('boxes_dens.pickle'      ,'rb'))
densities_data=pickle.load(open('densities_data.pickle'  ,'rb'))
qualities_dens=pickle.load(open('qualities_dens.pickle'  ,'rb'))
all_errors_dens=pickle.load(open('all_errors_dens.pickle' ,'rb'))

boxes_faz=pickle.load(open('boxes_faz_clean.pickle' ,'rb'))
faz_data=pickle.load(open('faz_data.pickle'        ,'rb'))
errors_faz=pickle.load(open('errors_faz.pickle'      ,'rb'))

# densities_data=data_dens
# all_errors_dens={ key:value for (key,value) in errors_dens.items() if value=='major' }


# COMPILE DF

In [25]:
meta={}
for idx,file in enumerate(files):
    match=re.search(r'P\d+', file)
    patient_number = match.group() if match else None
    
    match=re.search(r'V\d+', file)
    visit =match.group() if match else None
    
    match = re.search(r'(L|R) eye', file)  
    eye=match.group() if match else None
    
    match=re.search(r'\d+\.\d+', file)
    depth= match.group()   if match else None
    
    match=re.search(r'(DP|SUP|FAZ)', file)
    scantype=match.group() if match else None
    
    meta[idx]={'p':patient_number,
               'v':visit,
               'eye':eye,
               'dep':depth,
               'st':scantype}

In [26]:
proto={
    'p':np.nan,
    'v':np.nan,
    'eye':np.nan,
    'dep':np.nan,
    'st':np.nan,
    'QUALITY': np.nan,
    'depth_whole': np.nan,
    'depth_sup hemi': np.nan,
    'depth_inf hemi': np.nan,
    'depth_fovea': np.nan,
    'depth_parafovea': np.nan,
    'depth_para_sup hemi': np.nan,
    'depth_parai_inf hemi': np.nan,
    'depth_para temp': np.nan,
    'depth_para_sup': np.nan,
    'depth_para_nasal': np.nan,
    'depth_para_inf': np.nan,
    'depth_perifovea': np.nan,
    'depth_peri_sup hemi': np.nan,
    'depth_peri_inf hemi': np.nan,
    'depth_peri_tempo ': np.nan,
    'depth_peri_sup': np.nan,
    'depth_peri_nasal': np.nan,
    'depth_peri_inf': np.nan,
    'thick_whole': np.nan,
    'thick_sup hemi': np.nan,
    'thick_inf hemi': np.nan,
    'thick_fovea': np.nan,
    'thick_parafovea': np.nan,
    'thick_para_sup hemi': np.nan,
    'thick_parai_inf hemi': np.nan,
    'thick_para temp': np.nan,
    'thick_para_sup': np.nan,
    'thick_para_nasal': np.nan,
    'thick_para_inf': np.nan,
    'thick_perifovea': np.nan,
    'thick_peri_sup hemi': np.nan,
    'thick_peri_inf hemi': np.nan,
    'thick_peri_tempo ': np.nan,
    'thick_peri_sup': np.nan,
    'thick_peri_nasal': np.nan,
    'thick_peri_inf': np.nan,
    'grid_v_d1': np.nan,
    'grid_v_d2': np.nan,
    'grid_v_d3': np.nan,
    'grid_v_d4': np.nan,
    'grid_v_d5': np.nan,
    'grid_v_d6': np.nan,
    'grid_v_d7': np.nan,
    'grid_v_d8': np.nan,
    'grid_v_d9':np.nan,
    'area (mm2)':np.nan,
    'perim (mm)':np.nan,
    'FD':np.nan,
                }

In [27]:
pd.DataFrame(data=_,index=[0])

,p,v,eye,dep,st,QUALITY,depth_whole,depth_sup hemi,depth_inf hemi,depth_fovea,...,grid_v_d3,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9,area (mm2),perim (mm),FD
0,P104,V3,R eye,6.0,FAZ,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.288,2.107,59.23


In [28]:
%%time
major_error = {key: value for key, value in all_errors_dens.items() if value == 'major'}
indices_clean= list( set(list(densities_data.keys())+ list(faz_data.keys()))  - set(major_error.keys()) )
alldata=[]
for idx in indices_clean:
    _=proto.copy()
    _['p']   = meta[idx]['p']
    _['v']   = meta[idx]['v']
    _['eye'] = meta[idx]['eye']
    _['dep'] = meta[idx]['dep']
    _['st']  = meta[idx]['st']
    if _['st'] =='FAZ':
        _['area (mm2)'] = faz_data[idx]['faz'][0]
        _['perim (mm)'] = faz_data[idx]['faz'][1]
        _['FD']         = faz_data[idx]['faz'][2]
        _['QUALITY']    = faz_data[idx]['qual']
    else:
        _['QUALITY']= qualities_dens[idx]
        _['depth_whole'           ]  = densities_data[idx]['top_left'][0]
        _['depth_sup hemi'       ]   = densities_data[idx]['top_left'][1]
        _['depth_inf hemi'       ]   = densities_data[idx]['top_left'][2]
        _['depth_fovea'           ]  = densities_data[idx]['top_left'][3]
        _['depth_parafovea'       ]  = densities_data[idx]['top_left'][4]
        _['depth_para_sup hemi'  ]   = densities_data[idx]['top_left'][5]

        _['depth_parai_inf hemi' ]   = densities_data[idx]['top_left'][6]
        _['depth_para temp'      ]   = densities_data[idx]['top_left'][7]
        _['depth_para_sup'        ]  = densities_data[idx]['top_left'][8]
        _['depth_para_nasal'      ]  = densities_data[idx]['top_left'][9]
        _['depth_para_inf'        ]  = densities_data[idx]['top_left'][10]
        _['depth_perifovea'       ]  = densities_data[idx]['top_left'][11]
        _['depth_peri_sup hemi'  ]   = densities_data[idx]['top_left'][12]
        _['depth_peri_inf hemi'  ]   = densities_data[idx]['top_left'][13]
        _['depth_peri_tempo '     ]  = densities_data[idx]['top_left'][14]
        _['depth_peri_sup'        ]  = densities_data[idx]['top_left'][16]
        _['depth_peri_nasal'      ]  = densities_data[idx]['top_left'][16]
        _['depth_peri_inf'        ]  = densities_data[idx]['top_left'][17]
        _['thick_whole'           ]  = densities_data[idx]['top_right'][0]
        _['thick_sup hemi'       ]   = densities_data[idx]['top_right'][1]
        _['thick_inf hemi'       ]   = densities_data[idx]['top_right'][2]
        _['thick_fovea'           ]  = densities_data[idx]['top_right'][3]
        _['thick_parafovea'       ]  = densities_data[idx]['top_right'][4]
        _['thick_para_sup hemi'  ]   = densities_data[idx]['top_right'][5]
        _['thick_parai_inf hemi' ]   = densities_data[idx]['top_right'][6]
        _['thick_para temp'      ]   = densities_data[idx]['top_right'][7]
        _['thick_para_sup'        ]  = densities_data[idx]['top_right'][8]
        _['thick_para_nasal'      ]  = densities_data[idx]['top_right'][9]
        _['thick_para_inf'        ]  = densities_data[idx]['top_right'][10]
        _['thick_perifovea'       ]  = densities_data[idx]['top_right'][11]
        _['thick_peri_sup hemi'  ]   = densities_data[idx]['top_right'][12]
        _['thick_peri_inf hemi'  ]   = densities_data[idx]['top_right'][13]
        _['thick_peri_tempo '     ]  = densities_data[idx]['top_right'][14]
        _['thick_peri_sup'        ]  = densities_data[idx]['top_right'][16]
        _['thick_peri_nasal'      ]  = densities_data[idx]['top_right'][16]
        _['thick_peri_inf'        ]  = densities_data[idx]['top_right'][17]
        _['grid_v_d1'             ]  = densities_data[idx]['bottom'][0]
        _['grid_v_d2'             ]  = densities_data[idx]['bottom'][1]
        _['grid_v_d3'             ]  = densities_data[idx]['bottom'][2]
        _['grid_v_d4'             ]  = densities_data[idx]['bottom'][3]
        _['grid_v_d5'             ]  = densities_data[idx]['bottom'][4]
        _['grid_v_d6'             ]  = densities_data[idx]['bottom'][5]
        _['grid_v_d7'             ]  = densities_data[idx]['bottom'][6]
        _['grid_v_d8'             ]  = densities_data[idx]['bottom'][7]
        _['grid_v_d9'             ]  = densities_data[idx]['bottom'][8]
    
    alldata.append(_)
    
df=pd.DataFrame(alldata)
df['thick_whole']

CPU times: user 23.9 ms, sys: 0 ns, total: 23.9 ms
Wall time: 23.4 ms


0       307
1       307
2       NaN
3       269
4       269
       ... 
1583    322
1584    NaN
1585    289
1586    289
1587    NaN
Name: thick_whole, Length: 1588, dtype: object

In [29]:
ordered_cols=['p',
'v',
'eye',
'dep',
'st',
'QUALITY' ,
'depth_whole' ,
'depth_sup hemi' ,
'depth_inf hemi' ,
'depth_fovea' ,
'depth_parafovea' ,
'depth_para_sup hemi' ,
'depth_parai_inf hemi' ,
'depth_para temp' ,
'depth_para_sup' ,
'depth_para_nasal' ,
'depth_para_inf' ,
'depth_perifovea' ,
'depth_peri_sup hemi' ,
'depth_peri_inf hemi' ,
'depth_peri_tempo ' ,
'depth_peri_sup' ,
'depth_peri_nasal' ,
'depth_peri_inf' ,
'thick_whole' ,
'thick_sup hemi' ,
'thick_inf hemi' ,
'thick_fovea' ,
'thick_parafovea' ,
'thick_para_sup hemi' ,
'thick_parai_inf hemi' ,
'thick_para temp' ,
'thick_para_sup' ,
'thick_para_nasal' ,
'thick_para_inf' ,
'thick_perifovea' ,
'thick_peri_sup hemi' ,
'thick_peri_inf hemi' ,
'thick_peri_tempo ' ,
'thick_peri_sup' ,
'thick_peri_nasal' ,
'thick_peri_inf' ,
'grid_v_d1' ,
'grid_v_d2' ,
'grid_v_d3' ,
'grid_v_d4' ,
'grid_v_d5' ,
'grid_v_d6' ,
'grid_v_d7' ,
'grid_v_d8' ,
'grid_v_d9',
'area (mm2)',
'perim (mm)',
'FD']

df=df[ordered_cols]


# PIVOT ON DP AND SUP

In [30]:
#kinda works but ugly 
# df.iloc[0].fillna('')+df.iloc[1].fillna('')

#THIS IS THE SOLUTION. NP.NAN != NP.NAN , WHEREAS EVERYTHING ELSE == TO ITSELF. THEREFORE I USE THAT TO DISCRIMINATE
s='DP' #AGGREGATE EVERYTHING THAT IS NOT DP, so really SP
df_pivot_sup=pd.pivot_table(
    df[(df.st!=s)],
    index=["p",'v','dep','eye'],
    aggfunc=lambda x: x.iloc[0] if(x.iloc[0]==x.iloc[0] ) or (len(x)==1) else x.iloc[1]
              )
# df_pivot_sup=df_pivot_sup.apply(pd.to_numeric,errors='ignore')
# df_pivot_sup=df_pivot_sup.drop(['st'],axis=1)
df_pivot_sup=df_pivot_sup[ordered_cols[5:]]

s='SUP' #AGGREGATE EVERYTHING THAT IS NOT DP, so really SP
df_pivot_dp=pd.pivot_table(
    df[(df.st!=s)],
    index=["p",'v','dep','eye'],
    aggfunc=lambda x: x.iloc[0] if(x.iloc[0]==x.iloc[0] ) or (len(x)==1) else x.iloc[1]
              )
# df_pivot_dp=df_pivot_dp.apply(pd.to_numeric,errors='ignore')
# df_pivot_dp=df_pivot_dp.drop(['st'],axis=1)
df_pivot_dp=df_pivot_dp[ordered_cols[5:]]


### OR READ EM

In [20]:
df_pivot_sup =pd.read_csv('df_pivot_sup.csv')
df_pivot_dp  =pd.read_csv('df_pivot_sup.csv')
faz_v2       =pd.read_csv('faz_v2.csv')
faz_v3       =pd.read_csv('faz_v3.csv')

# COMPILE FAZ DFs

In [31]:
cols=['QUALITY','area (mm2)','perim (mm)','FD']

a_faz_v2=df_pivot_dp.xs('V2',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=lambda x:x).xs('L eye',level=1,axis=1
                                                                 ).xs('3.0',level=1,axis=1)[cols]
b_faz_v2=df_pivot_dp.xs('V2',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=lambda x:x).xs('L eye',level=1,axis=1
                                                                 ).xs('6.0',level=1,axis=1)[cols]
c_faz_v2=df_pivot_dp.xs('V2',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=lambda x:x).xs('R eye',level=1,axis=1
                                                                 ).xs('3.0',level=1,axis=1)[cols]
d_faz_v2=df_pivot_dp.xs('V2',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=lambda x:x).xs('R eye',level=1,axis=1
                                                                 ).xs('6.0',level=1,axis=1)[cols]
faz_v2=pd.concat([a_faz_v2,b_faz_v2,c_faz_v2,d_faz_v2],axis=1)
faz_v2=faz_v2.reset_index()
faz_v2

a_faz_v3=df_pivot_dp.xs('V3',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=lambda x:x).xs('L eye',level=1,axis=1
                                                                 ).xs('3.0',level=1,axis=1)[cols]
b_faz_v3=df_pivot_dp.xs('V3',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=lambda x:x).xs('L eye',level=1,axis=1
                                                                 ).xs('6.0',level=1,axis=1)[cols]
c_faz_v3=df_pivot_dp.xs('V3',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=lambda x:x).xs('R eye',level=1,axis=1
                                                                 ).xs('3.0',level=1,axis=1)[cols]
d_faz_v3=df_pivot_dp.xs('V3',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=lambda x:x).xs('R eye',level=1,axis=1
                                                                 ).xs('6.0',level=1,axis=1)[cols]
faz_v3=pd.concat([a_faz_v3,b_faz_v3,c_faz_v3,d_faz_v3],axis=1)
faz_v3=faz_v3.reset_index()

# DENSITY DP DATA

In [229]:
# cols=ordered_cols
cols=ordered_cols[5:-3]

In [239]:
dp_v2_piv=df_pivot_dp[cols].xs('V2',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=list )[cols]


dp_v2_piv=dp_v2_piv.applymap(lambda x: x[0] if type(x)==list else x)
dp_v2_piv
a_dp_v2=dp_v2_piv.xs('L eye',level=1,axis=1).xs('3.0',level=1,axis=1)
b_dp_v2=dp_v2_piv.xs('L eye',level=1,axis=1).xs('6.0',level=1,axis=1)
c_dp_v2=dp_v2_piv.xs('R eye',level=1,axis=1).xs('3.0',level=1,axis=1)
d_dp_v2=dp_v2_piv.xs('R eye',level=1,axis=1).xs('6.0',level=1,axis=1)
dp_v2=pd.concat([a_dp_v2[cols],b_dp_v2[cols],c_dp_v2[cols],d_dp_v2[cols]],axis=1)
dp_v2=dp_v2.reset_index()

dp_v3_piv=df_pivot_dp[cols].xs('V3',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=list )[cols]

dp_v3_piv=dp_v3_piv.applymap(lambda x: x[0] if type(x)==list else x)

a_dp_v3=dp_v3_piv.xs('L eye',level=1,axis=1).xs('3.0',level=1,axis=1)
b_dp_v3=dp_v3_piv.xs('L eye',level=1,axis=1).xs('6.0',level=1,axis=1)
c_dp_v3=dp_v3_piv.xs('R eye',level=1,axis=1).xs('3.0',level=1,axis=1)
d_dp_v3=dp_v3_piv.xs('R eye',level=1,axis=1).xs('6.0',level=1,axis=1)
dp_v3=pd.concat([a_dp_v3[cols],b_dp_v3[cols],c_dp_v3[cols],d_dp_v3[cols]],axis=1)
dp_v3=dp_v3.reset_index()



# DENSITY SUP

In [240]:
sup_v2_piv=df_pivot_sup[cols].xs('V2',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=list )[cols]

sup_v2_piv=sup_v2_piv.applymap(lambda x: x[0] if type(x)==list else x)

a_sup_v2=sup_v2_piv.xs('L eye',level=1,axis=1).xs('3.0',level=1,axis=1)
b_sup_v2=sup_v2_piv.xs('L eye',level=1,axis=1).xs('6.0',level=1,axis=1)
c_sup_v2=sup_v2_piv.xs('R eye',level=1,axis=1).xs('3.0',level=1,axis=1)
d_sup_v2=sup_v2_piv.xs('R eye',level=1,axis=1).xs('6.0',level=1,axis=1)
sup_v2=pd.concat([a_sup_v2[cols],b_sup_v2[cols],c_sup_v2[cols],d_sup_v2[cols]],axis=1)
sup_v2=sup_v2.reset_index()

sup_v3_piv=df_pivot_sup[cols].xs('V3',level=1).pivot_table(index='p',
                                           columns=['eye','dep'],
                                           aggfunc=list )[cols]

sup_v3_piv=sup_v3_piv.applymap(lambda x: x[0] if type(x)==list else x)

a_sup_v3=sup_v3_piv.xs('L eye',level=1,axis=1).xs('3.0',level=1,axis=1)
b_sup_v3=sup_v3_piv.xs('L eye',level=1,axis=1).xs('6.0',level=1,axis=1)
c_sup_v3=sup_v3_piv.xs('R eye',level=1,axis=1).xs('3.0',level=1,axis=1)
d_sup_v3=sup_v3_piv.xs('R eye',level=1,axis=1).xs('6.0',level=1,axis=1)
sup_v3=pd.concat([a_sup_v3[cols],b_sup_v3[cols],c_sup_v3[cols],d_sup_v3[cols]],axis=1)
sup_v3=sup_v3.reset_index()
ordered_merged=sup_v3.columns

# ALL COLLATED

In [241]:
collated_v2=pd.concat([
    a_dp_v2,a_sup_v2,a_faz_v2,
    b_dp_v2,b_sup_v2,b_faz_v2,
    c_dp_v2,c_sup_v2,c_faz_v2,
    d_dp_v2,d_sup_v2,d_faz_v2,
                      ],axis=1)
collated_v2=collated_v2.reset_index()

collated_v3=pd.concat([
    a_dp_v3,a_sup_v3,a_faz_v3,
    b_dp_v3,b_sup_v3,b_faz_v3,
    c_dp_v3,c_sup_v3,c_faz_v3,
    d_dp_v3,d_sup_v3,d_faz_v3,
                      
                      ],axis=1,)
collated_v3=collated_v3.reset_index()
collated_v3.columns.values

array(['p', 'QUALITY', 'depth_whole', 'depth_sup hemi', 'depth_inf hemi',
       'depth_fovea', 'depth_parafovea', 'depth_para_sup hemi',
       'depth_parai_inf hemi', 'depth_para temp', 'depth_para_sup',
       'depth_para_nasal', 'depth_para_inf', 'depth_perifovea',
       'depth_peri_sup hemi', 'depth_peri_inf hemi', 'depth_peri_tempo ',
       'depth_peri_sup', 'depth_peri_nasal', 'depth_peri_inf',
       'thick_whole', 'thick_sup hemi', 'thick_inf hemi', 'thick_fovea',
       'thick_parafovea', 'thick_para_sup hemi', 'thick_parai_inf hemi',
       'thick_para temp', 'thick_para_sup', 'thick_para_nasal',
       'thick_para_inf', 'thick_perifovea', 'thick_peri_sup hemi',
       'thick_peri_inf hemi', 'thick_peri_tempo ', 'thick_peri_sup',
       'thick_peri_nasal', 'thick_peri_inf', 'grid_v_d1', 'grid_v_d2',
       'grid_v_d3', 'grid_v_d4', 'grid_v_d5', 'grid_v_d6', 'grid_v_d7',
       'grid_v_d8', 'grid_v_d9', 'QUALITY', 'depth_whole',
       'depth_sup hemi', 'depth_inf hemi', 'd

### SAVE EM

In [242]:
df_pivot_dp.to_csv('df_pivot_dp.csv')
df_pivot_sup.to_csv('df_pivot_sup.csv')
faz_v2.to_csv('faz_v2.csv')
faz_v3.to_csv('faz_v3.csv')

# ERRORS HIGHLIGHTING

In [249]:
import re
from collections import Counter

highlights_v2={}
for col_n in range(1,len(collated_v2.columns)):
    col=collated_v2.iloc[:,col_n].fillna('N/A')
    pattern=infer_regex(col.values)
    matches = col.astype(str).str.match(pattern)
    rows_match=col[matches==False]
    if len(rows_match[rows_match != 'N/A'])>0:
        highlights_v2[col_n]=rows_match[rows_match != 'N/A'].index.values
    
highlights_v3={}
for col_n in range(1,len(collated_v2.columns)):
    col=collated_v3.iloc[:,col_n].fillna('N/A')
    pattern=infer_regex(col.values)
    matches = col.astype(str).str.match(pattern)
    rows_match=col[matches==False]
    if len(rows_match[rows_match != 'N/A'])>0:
        highlights_v3[col_n]=rows_match[rows_match != 'N/A'].index.values
    


Counter({'d': 76, 'N/A': 3})
Counter({'dd.d': 72, 'N/A': 4, 'dd.': 3})
Counter({'dd.d': 73, 'N/A': 4, 'dd.': 2})
Counter({'dd.d': 73, 'N/A': 4, 'dd.': 2})
Counter({'dd.d': 74, 'N/A': 4, 'dd.': 1})
Counter({'dd.d': 71, 'N/A': 4, 'dd.': 3, 'Nddd': 1})
Counter({'dd.d': 72, 'N/A': 4, 'dd.': 3})
Counter({'dd.d': 73, 'N/A': 5, 'dd.': 1})
Counter({'dd.d': 74, 'N/A': 4, 'dd.': 1})
Counter({'dd.d': 74, 'N/A': 4, 'dd.': 1})
Counter({'dd.d': 73, 'N/A': 5, 'dd.': 1})
Counter({'dd.d': 73, 'N/A': 5, 'dd.': 1})
Counter({'N/A': 79})
Counter({'N/A': 79})
Counter({'N/A': 79})
Counter({'N/A': 79})
Counter({'N/A': 79})
Counter({'N/A': 79})
Counter({'N/A': 79})
Counter({'ddd': 73, 'N/A': 4, 'dd': 2})
Counter({'ddd': 75, 'N/A': 4})
Counter({'ddd': 74, 'N/A': 5})
Counter({'ddd': 75, 'N/A': 4})
Counter({'ddd': 75, 'N/A': 4})
Counter({'ddd': 75, 'N/A': 4})
Counter({'ddd': 74, 'N/A': 5})
Counter({'ddd': 75, 'N/A': 4})
Counter({'ddd': 75, 'N/A': 4})
Counter({'ddd': 74, 'N/A': 5})
Counter({'ddd': 74, 'N/A': 5})
C

Counter({'N/A': 79})
Counter({'N/A': 79})
Counter({'dd.d': 52, 'N/A': 24, 'dd.': 3})
Counter({'dd.d': 52, 'N/A': 24, 'dd.': 3})
Counter({'dd.d': 54, 'N/A': 24, 'dd.': 1})
Counter({'dd.d': 55, 'N/A': 24})
Counter({'dd.d': 55, 'N/A': 24})
Counter({'dd.d': 55, 'N/A': 24})
Counter({'dd.d': 54, 'N/A': 24, 'dd.dd': 1})
Counter({'dd.d': 55, 'N/A': 24})
Counter({'dd.d': 55, 'N/A': 24})
Counter({'d': 79})
Counter({'d.ddd': 77, 'N/A': 2})
Counter({'d.ddd': 77, 'N/A': 2})
Counter({'dd.dd': 72, 'dddd': 5, 'N/A': 2})
Counter({'d': 75, 'N/A': 4})
Counter({'dd.d': 71, 'N/A': 6, 'dd.': 2})
Counter({'dd.d': 73, 'N/A': 6})
Counter({'dd.d': 73, 'N/A': 6})
Counter({'dd.d': 73, 'N/A': 6})
Counter({'dd.d': 71, 'N/A': 6, 'dd.': 2})
Counter({'dd.d': 71, 'N/A': 6, 'dd.': 2})
Counter({'dd.d': 72, 'N/A': 6, 'dd.': 1})
Counter({'dd.d': 72, 'N/A': 6, 'dd.': 1})
Counter({'dd.d': 73, 'N/A': 6})
Counter({'dd.d': 73, 'N/A': 6})
Counter({'dd.d': 73, 'N/A': 6})
Counter({'dd.d': 70, 'N/A': 7, 'dd.': 2})
Counter({'dd.d': 

Counter({'ddd': 49, 'N/A': 28})
Counter({'ddd': 49, 'N/A': 28})
Counter({'ddd': 49, 'N/A': 28})
Counter({'ddd': 49, 'N/A': 28})
Counter({'ddd': 49, 'N/A': 28})
Counter({'dd.d': 46, 'N/A': 28, 'dd.': 3})
Counter({'dd.d': 48, 'N/A': 28, 'dd.': 1})
Counter({'dd.d': 49, 'N/A': 28})
Counter({'dd.d': 49, 'N/A': 28})
Counter({'dd.d': 49, 'N/A': 28})
Counter({'dd.d': 49, 'N/A': 28})
Counter({'dd.d': 45, 'N/A': 28, 'dd.dd': 4})
Counter({'dd.d': 49, 'N/A': 28})
Counter({'dd.d': 49, 'N/A': 28})
Counter({'d': 71, 'N/A': 6})
Counter({'d.ddd': 70, 'N/A': 7})
Counter({'d.ddd': 70, 'N/A': 7})
Counter({'dd.dd': 70, 'N/A': 7})
Counter({'d': 72, 'N/A': 5})
Counter({'dd.d': 71, 'N/A': 6})
Counter({'dd.d': 71, 'N/A': 6})
Counter({'dd.d': 71, 'N/A': 6})
Counter({'dd.d': 71, 'N/A': 6})
Counter({'dd.d': 70, 'N/A': 7})
Counter({'dd.d': 69, 'N/A': 7, 'dd.': 1})
Counter({'dd.d': 69, 'N/A': 8})
Counter({'dd.d': 71, 'N/A': 6})
Counter({'dd.d': 69, 'N/A': 7, 'dd.': 1})
Counter({'dd.d': 70, 'N/A': 7})
Counter({'dd.d

In [243]:
collated_v3[collated_v3.p=='P045'].iloc[:,143:183]

,QUALITY,depth_whole,depth_sup hemi,depth_inf hemi,depth_fovea,depth_parafovea,depth_para_sup hemi,depth_parai_inf hemi,depth_para temp,depth_para_sup,...,thick_perifovea,thick_peri_sup hemi,thick_peri_inf hemi,thick_peri_tempo,thick_peri_sup,thick_peri_nasal,thick_peri_inf,grid_v_d1,grid_v_d2,grid_v_d3
35,7,53.4,52.4,54.5,33.4,56.3,55.4,57.3,56.6,54.9,...,308,311,304,293,327,327,297,58.9,52.7,45.1


In [244]:
df_pivot_sup.loc[('P045','V3','6.0')]


,QUALITY,depth_whole,depth_sup hemi,depth_inf hemi,depth_fovea,depth_parafovea,depth_para_sup hemi,depth_parai_inf hemi,depth_para temp,depth_para_sup,...,grid_v_d3,grid_v_d4,grid_v_d5,grid_v_d6,grid_v_d7,grid_v_d8,grid_v_d9,area (mm2),perim (mm),FD
eye,,,,,,,,,,,,,,,,,,,,,
L eye,7,53.4,52.4,54.5,33.4,56.3,55.4,57.3,56.6,54.9,...,45.1,57.2,51.9,51.6,59.9,55.6,47.8,0.135,1.419,54.68
R eye,8,52.2,52.4,52.1,30.0,54.0,54.2,53.9,54.5,54.1,...,56.8,50.6,49.2,57.3,47.3,51.6,56.1,0.122,1.356,51.37


In [190]:
# collated_v3.loc[collated_v3.p=='P045','QUALITY'].
np.where(collated_v3.columns.get_loc('QUALITY'))

(array([  1,  47,  93,  97, 143, 189, 193, 239, 285, 289, 335, 381]),)

In [197]:
collated_v3[collated_v3.p=='P045'].iloc[:,143:183].values

array([['7', '33.4', '54.5', '56.6', '57.7', '56.1', '54.9', '55.4',
        '56.3', '57.3', '55.1', '54.2', '71.', '71.', '52.9', '48.3',
        '53.6', '52.4', '53.4', '58.9', '52.7', '45.1', '57.2', '51.9',
        '51.6', '59.9', '55.6', '47.8', '285', '308', '329', '345',
        '350', '349', '344', '343', '342', '297', '304', '327']],
      dtype=object)

# EXCEL WRITING

In [250]:
from openpyxl.utils.dataframe import dataframe_to_rows
wb=op.load_workbook('./FORM_OCTA.xlsx')

In [251]:
def unmerge_18(ws):
    for merge in list(ws.merged_cells):
        if merge.min_row>2:
            ws.unmerge_cells(range_string=str(merge))
            
sheet_names=wb.sheetnames

##################################################################### V2_COLLATED
wb.active=wb['V2 COLLATED']
ws=wb.active
unmerge_18(ws)
rows = dataframe_to_rows(collated_v2,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)
##################################################################### V3_COLLATED
wb.active=wb['V3 COLLATED']
ws=wb.active
unmerge_18(ws)
rows = dataframe_to_rows(collated_v3,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)
            
# ###################################################################### V2_DENSITY_DP
wb.active=wb[ 'V2 DENSITY DP DATA ']
ws=wb.active
unmerge_18(ws)
rows = dataframe_to_rows(dp_v2,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)

###################################################################### V3_DENSITY_DP
wb.active=wb[ 'V3 DENSITY DP DATA']
ws=wb.active
unmerge_18(ws)
rows = dataframe_to_rows(dp_v3,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)
###################################################################### V2_DENSITY_SUP
wb.active=wb[ 'V2 DENSITY SUP DATA']
ws=wb.active
unmerge_18(ws)
rows = dataframe_to_rows(sup_v2,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)
###################################################################### V3_DENSITY_SUP
wb.active=wb[ 'V3 DENSITY SUP DATA']
ws=wb.active
unmerge_18(ws)
rows = dataframe_to_rows(sup_v3,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)
            
###################################################################### V2_FAZ
wb.active=wb[ 'V2 FAZ']
ws=wb.active
unmerge_18(ws)
rows = dataframe_to_rows(faz_v2,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)
###################################################################### V3_FAZ
wb.active=wb[ 'V3 FAZ']
ws=wb.active
unmerge_18(ws)
rows = dataframe_to_rows(faz_v3,index=False,header=False)
for r_idx, row in enumerate(rows, 4):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)


## HIGHLIGHTING

In [252]:
from openpyxl.styles import PatternFill
yellow_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

### V2
wb.active=wb['V2 COLLATED']
ws=wb.active
for col in highlights_v2:
    for row in highlights_v2[col]:
        ws.cell(row=row+4, column=col+1).fill = yellow_fill

### V3
wb.active=wb['V3 COLLATED']
ws=wb.active
for col in highlights_v3:
    for row in highlights_v3[col]:
        ws.cell(row=row+4, column=col+1).fill = yellow_fill
        


In [253]:
wb.save("OCTA_EXTRACTION_HIGHLIGHTL.xlsx")

379

# ERROR ESTIMATION

In [259]:
count_highl=0
for i in highlights_v2:
    count_highl+=len(highlights_v2[i])
for i in highlights_v3:
    count_highl+=len(highlights_v3[i])

error_percent=count_highl*100/(collated_v2.count().sum()+collated_v3.count().sum())
print(f'there are {count_highl} errors, over {(collated_v2.count().sum()+collated_v3.count().sum())} cells, corresponding to a {error_percent}% ')

there are 379 errors, over 48185 cells, corresponding to a 0.7865518314828266% 
